## Download the LLM

In [3]:
import os
from huggingface_hub import hf_hub_download

create access token on huggingFace via profile settings and set in your system user variables 

In [4]:
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY") 

get model 
[a local copy of mistral](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/tree/main) 

In [9]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
filenames = [ "config.json","generation_config.json","model-00001-of-00003.safetensors","model-00002-of-00003.safetensors","model-00003-of-00003.safetensors",
              "model.safetensors.index.json","pytorch_model-00001-of-00003.bin","pytorch_model-00002-of-00003.bin","pytorch_model-00003-of-00003.bin",
              "pytorch_model.bin.index.json","special_tokens_map.json","tokenizer.json","tokenizer.model","tokenizer_config.json"]


In [ ]:
for filename in filenames: 
    downloaded_model_path = hf_hub_download(
            repo_id = model_id,
            filename = filename,
            token = HUGGING_FACE_API_KEY
    )
    print (downloaded_model_path)

C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2\snapshots\cf47bb3e18fe41a5351bc36eef76e9c900847c89\config.json
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2\snapshots\cf47bb3e18fe41a5351bc36eef76e9c900847c89\generation_config.json
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2\snapshots\cf47bb3e18fe41a5351bc36eef76e9c900847c89\model-00001-of-00003.safetensors
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2\snapshots\cf47bb3e18fe41a5351bc36eef76e9c900847c89\model-00002-of-00003.safetensors
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2\snapshots\cf47bb3e18fe41a5351bc36eef76e9c900847c89\model-00003-of-00003.safetensors
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2\snapshots\cf47bb3e18fe41a5351bc36eef76e9c900847

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.94G [00:00<?, ?B/s]